In [1]:
!pip install polars


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
# convert .csv to .parquet

import glob

pths = glob.glob('data/*.csv')

import polars as pl

for pth in pths:
    _name = pth.split(".")[0] + ".parquet"
    _df = pl.scan_csv(pth).collect().write_parquet(_name)

In [7]:
orders_pth = 'data/orders.parquet'

orders = pl.scan_parquet(orders_pth)

print(f"Total orders rows: {orders.collect().shape[0]:,}")

Total orders rows: 3,421,083


In [9]:
print(f"The unique numeber of customer is {orders.select("user_id").unique().collect().shape[0]:,}")

The unique numeber of customer is 206,209


In [10]:
products_pth = 'data/products.parquet'

products = pl.scan_parquet(products_pth)

print(f"There are {products.select("product_id").unique().collect().shape[0]:,} unique products.")

There are 49,688 unique products.


In [11]:
order_products__prior_pth = 'data/order_products__prior.parquet'
order_items = pl.scan_parquet(order_products__prior_pth)

In [18]:
top5 = order_items.select(pl.col("product_id").value_counts(sort=True)).head(5).collect().unnest("product_id")

In [27]:
print("\n Top 5 is:\n", top5.with_columns(product_name=pl.col("product_id").replace_strict(dict(zip(products.collect()["product_id"], products.collect()["product_name"])))))


 Top 5 is:
 shape: (5, 3)
┌────────────┬────────┬────────────────────────┐
│ product_id ┆ count  ┆ product_name           │
│ ---        ┆ ---    ┆ ---                    │
│ i64        ┆ u32    ┆ str                    │
╞════════════╪════════╪════════════════════════╡
│ 24852      ┆ 472565 ┆ Banana                 │
│ 13176      ┆ 379450 ┆ Bag of Organic Bananas │
│ 21137      ┆ 264683 ┆ Organic Strawberries   │
│ 21903      ┆ 241921 ┆ Organic Baby Spinach   │
│ 47209      ┆ 213584 ┆ Organic Hass Avocado   │
└────────────┴────────┴────────────────────────┘
